## Create Profiles Table
First we'll create table to store our customer profiles. We'll use the information in this table to personalize the emails that we send. In order to use this data with a Census Live Sync, we need to enable change tracking. (Note: change tracking can only be enabled by the table's owner, so it's easiest to configure this when the table is first created.)

In [ ]:
CREATE OR REPLACE TABLE customer_profiles (
    customer_id INT PRIMARY KEY,
    name VARCHAR,
    email_address VARCHAR
)
CHANGE_TRACKING = TRUE;

## Create Events Table
Next, we'll create a table to store event data related to our customers. We'll use these events to trigger our emails (emails will be sent when new events are created). Events will be associated with a customer by ID, and we can use this to associate them with the customer profile data we already have. Change tracking needs to be enabled for this table as well.

In [ ]:
CREATE OR REPLACE TABLE customer_events (
    id BIGINT PRIMARY KEY,
    customer_id INT,
    time TIMESTAMP_TZ NOT NULL,
    event_type VARCHAR NOT NULL,
    order_number INT
)
CHANGE_TRACKING = TRUE;

## Join and Filter Profile and Event Data in a Dynamic Table
Now that we have tables for both our profile and event data, we'll join and filter them to create the data we want to use to send emails by creating a dynamic table. The dynamic table is incrementally refreshed when the data in the tables referenced in the query are updated. Our dynamic table has a target lag of 1 minute, which means new events will be added within about a minute of when they are created in the events table.

We're filtering our events by type. This allows us to use one events table to store many different kinds of customer-related events, and then define in the dynamic table's query which events we want to use to trigger our emails.

Each row in this dynamic table represents a single email we want to send to a customer related to a specific event.

In [ ]:
CREATE DYNAMIC TABLE order_delivered_email_sends (
    event_id,
    event_time,
    customer_name,
    customer_email_address,
    order_number
)
TARGET_LAG = '1 minutes'
WAREHOUSE = DEVELOPER_TESTING
REFRESH_MODE = INCREMENTAL
AS SELECT
    customer_events.id,
    customer_events.time,
    customer_profiles.name,
    customer_profiles.email_address,
    customer_events.order_number
    FROM customer_events
        INNER JOIN customer_profiles
            ON customer_events.customer_id = customer_profiles.customer_id
    WHERE customer_events.event_type = 'OrderDelivered'
        AND customer_profiles.email_address IS NOT NULL;

## Create a Census Live Sync

Now we're ready to create a Census Live Sync that will send emails via our email service provider or marketing automation tool based on the data in our dynamic table.

### Prerequisites
- If you haven't already, [start a free Census trial](https://app.getcensus.com/signup).
- [Connect your Snowflake account to Census as a data source](https://docs.getcensus.com/sources/available-sources/snowflake).
- Connect your email service provider or marketing automation tool to Census as a destination. For this example, we'll be using [Braze](https://docs.getcensus.com/destinations/available-destinations/braze).

### Create the Sync
Using the navigation menu on the left side of [Census](https://app.getcensus.com), navigate to **Syncs**, then click the **New Sync** button in the upper right corner.
The New Sync page appears.

1. Under **Select a Source**, pick **Any Warehouse Table**, select your Snowflake connection, and select your database, schema, and the `order_delivered_email_sends` dynamic table.
2. Under **Select a Destination**, select your Braze connection, and select an existing Braze API-triggered campaign or API-triggered canvas.
3. Under **Select a Sync Behavior**, choose **Activate Events**. This ensures that emails are only sent when new records are added. Then choose **Send**.
4. Under **Select as Sync Key**, choose your `customer_id` column, and map this to the Braze **External ID** field.
5. Map additional columns to custom attributes in Braze. These attributes can be used to customize the email template that your send. For example, you can greet the customer by name by mapping the `customer_name` column, and refer to their order number using the `order_number` column.
6. Click **Next** to proceed to the Sync Summary page.
7. Under **Select a Run Mode**, choose **Live**. Census will create and continuously monitor a [Snowflake Stream](https://docs.snowflake.com/en/user-guide/streams-intro) to track and activate changes to your dynamic table as they occur.
8. Click **Create**, then, on the Sync Details page, click the toggle in the upper right corner of the page to enable your Live Sync.

## Create a Customer Profile
Once the Census Live Sync is running, it will take a few minutes to start up and provision a Snowflake Stream in your warehouse to begin monitoring changes. While the sync is starting, we can create a customer profile.

If you're using Braze, you'll need a corresponding User in Braze with a matching external ID. If you're using an email service provider like [SendGrid](https://docs.getcensus.com/destinations/available-destinations/sendgrid), you may be able to simply identify customers by email address.

We're using test data here, but you will want to ensure the email is sent to an email address you have access to.

In [ ]:
INSERT INTO customer_profiles (customer_id, name, email_address) VALUES (123, 'Alice', 'alice@example.com');

## Create an Order Delivered Event

The total time it takes for the data to reach Census and be sent on to your email service provider or marketing automation tool is dependent on the target lag of your dynamic table. As soon as Census receives your data, the counters on the Sync Details screen update. However, it can take a minute or two for an email to actually reach your recipient's inbox depending on both your email service provider, and your recipient's email host.

In [ ]:
INSERT INTO customer_events (id, time, type, customer_id, order_number) VALUES (1, NOW(), 'OrderDelivered', 123, 999);

## Access the Census Sync Log in Snowflake

As long as we are sending email to ourselves, it's easy to keep track of which emails have been sent and received. Once you're are sending email to your customers, you'll want observability into what data Census has received, when and how it activated that data, and information about any errors or failures that have occurred.

Census Live Syncs write sync logs to the `CENSUS.CENSUS` schema in your Snowflake account. You can query the `SYNC_LOG` view to access information about all your Census syncs.

If you look in the Census schema, you will also find the Snowflake Stream that Census created when you created your Live Sync. Do not read from this stream yourself, because reading from a stream consumes the data inside it, preventing Census from being able to activate the data for your.

Note that new entries in your sync log from a Census Live Sync may take seconds or minutes to appear.

In [ ]:
SELECT * FROM CENSUS.CENSUS.SYNC_LOG LIMIT 100;